In [32]:
from core.libs import pd
from core.db import get_engine
from core.schema_helpers import rename_columns_using_schema
from CruisesProcessor.general_importer import prepare_df_for_sql, save_inventory_to_sql, ensure_table
from CruisesProcessor.dead_alive_calculator import calculate_dead_alive
from core.doyle_calculator import calculate_doyle

archivo = r"C:\Users\HeyCe\World Tree Technologies Inc\Forest Inventory - Documentos\Mexico\2021_ForestInventory\6-Database\2021_Mexico_Database v1.0.xlsx"
sheet = 'TreeList_Data'

In [38]:
#Leer el archivo
df_original = pd.read_excel(archivo, sheet_name=sheet)
print("📄 Columnas originales:", list(df_original.columns))

df = df_original.drop(columns=["DAP (cm)", "THt (m)", "MHt (m)"], errors="ignore")
print("📄 Columnas actuales:  ", list(df.columns))

# Mostrar diferencia
eliminadas = set(df_original.columns) - set(df.columns)
print("🗑️ Columnas eliminadas:", eliminadas)
df = rename_columns_using_schema(df)
print(df.head(1))


📄 Columnas originales: ['FarmerCode', 'Contract', 'Property', 'PlantingYear', 'PropertyID', 'StandID', 'Plot#', 'TreeID', 'TPA_1/10ac', 'TreeStatus', 'DBH (in)', 'THt (ft)', 'MHt (ft)', 'DAP (cm)', 'THt (m)', 'MHt (m)', 'BA (sqft/ac)', 'BA_ac', 'VOL_m3', 'VOL_m3ac', 'MBF', 'MBF_ac']
📄 Columnas actuales:   ['FarmerCode', 'Contract', 'Property', 'PlantingYear', 'PropertyID', 'StandID', 'Plot#', 'TreeID', 'TPA_1/10ac', 'TreeStatus', 'DBH (in)', 'THt (ft)', 'MHt (ft)', 'BA (sqft/ac)', 'BA_ac', 'VOL_m3', 'VOL_m3ac', 'MBF', 'MBF_ac']
🗑️ Columnas eliminadas: {'THt (m)', 'MHt (m)', 'DAP (cm)'}
   FarmerCode contractcode         farmername  PlantingYear  PropertyID  \
0     40003.0       MX0003  AnaPaulinaHurtado        2021.0         NaN   

   StandID  plot  tree_number  TPA_1/10ac Status  dbh_in  tht_ft  merch_ht_ft  \
0      NaN   1.0          1.0        10.0   Live     1.0     1.0          NaN   

   BA (sqft/ac)    BA_ac    VOL_m3  VOL_m3ac       MBF    MBF_ac  
0      0.005454  0.05454  

In [39]:
# CALCULADORES que faltaban:
engine = get_engine()
df = calculate_dead_alive(df, engine)
df = calculate_doyle(df)

💻 Conectado a la base de datos helloworldtree
🌳 Doyle calculado


In [40]:
df_sql, _ = prepare_df_for_sql(df)
df_sql = df_sql.where(pd.notna(df_sql), None)

# ← El segundo valor (dtypes) lo ignoramos
tabla_destino = "inventory_mx_2022"

In [41]:
ensure_table(df_sql, engine, tabla_destino)

In [42]:
save_inventory_to_sql(df_sql, engine, tabla_destino)




=== INICIO DE IMPORTACIÓN ===
💻 Conectado a la base de datos helloworldtree
✅ Bulk insert completado: 
 'inventory_mx_2022' (12432 filas)
